# 8.1 读写包

<ul>
    <li>OpenPyXL</li>
    <li>XlsxWriter</li>
    <li>pyxlsx</li>
    <li>xlrd</li>
    <li>xlwt</li>
    <li>xlutils</li>
</ul>

<table align="left" width="500px" height="300px">
    <tr>
        <th>Excel文件格式</th>
        <th>读</th>
        <th>写</th>
        <th>编辑</th>
    </tr>
    <tr>
        <td>xlsx</td>
        <td>OpenPyXL</td>
        <td>OpenPyXL,XlsaWriter</td>
        <td>OpenPyXL</td>
    </tr>
    <tr>
        <td>xlsm</td>
        <td>OpenPyXL</td>
        <td>OpenPyXL,XlsaWriter</td>
        <td>OpenPyXL</td>
    </tr>
    <tr>
        <td>xltx，xltm</td>
        <td>OpenPyXL</td>
        <td>OpenPyXL</td>
        <td>OpenPyXL</td>
    </tr>
    <tr>
        <td>xlsb</td>
        <td>pyxlsb</td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>xls，xlt</td>
        <td>xlrd</td>
        <td>xlwt</td>
        <td>xlutils</td>
    </tr>
</table>

# OpenPyXL 【用于xlsx文件】
【单元格索引从1开始】

1. 使用OpenPyXL读取文件

In [1]:
import pandas as pd
import openpyxl
import excel
import datetime as dt

In [2]:
# 打开工作簿来读取单元格的值【使用data_only=True参数打开工作簿，默认值是False】
# 在加载数据之后文件会自动关闭
book = openpyxl.load_workbook("xl/stores.xlsx", data_only=True)

In [3]:
# 通过名称或索引（从0开始）获取工作表对象
sheet = book["2019"]
sheet = book.worksheets[0]

In [4]:
# 获取所有工作表名称的列表
book.sheetnames

['2019', '2020', '2019-2020']

In [5]:
# 遍历所有工作表对象
# openpyxl使用的是title而不是name
for i in book.worksheets:
    print(i.title)

2019
2020
2019-2020


In [6]:
# 获取维度，以工作表所选区域为例
sheet.max_row, sheet.max_column

(8, 6)

In [7]:
# 读取单个单元格的值，分别使用的是A1这种表示法，以及单元格索引（从1开始）
sheet["B6"].value
sheet.cell(row=6, column=2).value

'Boston'

In [8]:
# 使用excel模块来读取一个单元格区域的值
data = excel.read(book["2019"], (2, 2), (8, 6))
# 打印前两行
data[:2]

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

2. 使用OpenPyXL写入文件

In [9]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, colors
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.fills import PatternFill
import excel

In [10]:
# 实例化工作簿
book = openpyxl.Workbook()

# 获取第一张工作表并赋予它一个名称
sheet = book.active
sheet.title = "Sheet1"

# 使用A1表示法和单元格索引（从1开始）写入各个单元格
sheet["A1"].value = "Hello 1"
sheet.cell(row=2, column=1, value="Hello 2")

# 格式化：填充颜色、对齐、边框和字体
font_format = Font(color="FF0000", bold=True)
thin = Side(border_style="thin", color="FF0000")
sheet["A3"].value = "Hello 3"
sheet["A3"].font = font_format
sheet["A3"].border = Border(top=thin, left=thin, 
                            right=thin, bottom=thin)
sheet["A3"].alignment = Alignment(horizontal="center")
sheet["A3"].fill = PatternFill(fgColor="FFFF00", fill_type="solid")

# 数字格式化（使用Excel的格式化字符串）
sheet["A4"].value = 3.3333
sheet["A4"].number_format = "0.00"

# 日期格式化（使用Excel的格式化字符串）
sheet["A5"].value = dt.date(2016, 10, 13)
sheet["A5"].number_format = "mm/dd/yy"

# 公式：必须使用以逗号分隔的英文公式名称
sheet["A6"].value = "=SUM(A4, 2)"

# 图片
sheet.add_image(Image("images/python.png"), "C1")

# 二维列表（使用Excel模块）
data = [[None, "North", "South"], 
        ["Last Year", 2, 5], 
        ["This Year", 3, 6]]
excel.write(sheet, data, "A10")

# 图表
chart = BarChart()
chart.type = "col"
chart.title = "Sales Per Region"
chart.x_axis.title = "Regions"
chart.y_axis.title = "Sales"
chart_data = Reference(sheet, min_row=11, min_col=1, max_row=12, max_col=3)
chart_categories = Reference(sheet, min_row=10, min_col=2, max_row=10, max_col=3)
# from_rows就像你手动在Excel中
# 添加图表那样解释数据
chart.add_data(chart_data, titles_from_data=True, from_rows=True)
chart.set_categories(chart_categories)
sheet.add_chart(chart, "A15")

# 保存工作簿会在磁盘上创建文件
book.save("xl/openpyxl.xlsx")

若想写入为Excel模版文件，则需要再保存前设置template属性为True

In [11]:
book = openpyxl.Workbook()
sheet = book.active
sheet["A1"].value = "This is a template"
book.template = True
book.save("xl/template.xltx")

3. 使用OpenPyXL编辑文件

In [12]:
# 读取stores.xlsx文件，修改一个单元格，并将其以新的名称保存到新的位置
book = openpyxl.load_workbook("xl/stores.xlsx")
book["2019"]["A1"].value = "modified"
book.save("xl/stores_edited.xlsx")

若想写入为xlsm文件，那么OpenPyXL就必须处理一个已经存在的文件，并且在加载时需要将keep_vba参数设置为True

In [13]:
book = openpyxl.load_workbook("xl/macro.xlsm", keep_vba=True)
book["Sheet1"]["A1"].value = "Click the buttom!"
book.save("xl/macro_openpyxl.xlsm")

# XlsxWriter
【单元格索引从0开始】

In [14]:
import datetime as dt
import xlsxwriter
import excel

In [15]:
# 实例化工作簿
book = xlsxwriter.Workbook("xl/xlsxwriter.xlsx")

# 添加工作表并为其命名
sheet = book.add_worksheet("Sheet1")

# 使用A1表示法和单元格索引（从0开始）写入各个单元格
sheet.write("A1", "Hello 1")
sheet.write(1, 0, "Hello 2")

# 格式化：填充颜色、对齐、边框和字体
formatting = book.add_format({"font_color": "#FF0000", 
                              "bg_color": "#FFFF00", 
                              "bold": True, "align": "center", 
                              "border": 1, "border_color": "#FF0000"})
sheet.write("A3",  "Hello 3", formatting)

# 数字格式化（使用Excel的格式化字符串）
number_format = book.add_format({"num_format": "0.00"})
sheet.write("A4", 3.3333, number_format)

# 日期格式化（使用Excel的格式化字符串）
date_format = book.add_format({"num_format": "mm/dd/yy"})
sheet.write("A5", dt.date(2016, 10, 13), date_format)

# 公式
sheet.write("A6", "=SUM(A4, 2)")

# 图片
sheet.insert_image(0, 2, "images/python.png")

# 二维列表（使用excel模块）
data = [[None, "North", "South"], 
        ["Last Year", 2, 5], 
        ["This Year", 3, 6]]
excel.write(sheet, data, "A10")

# 图表【使用索引而不是单元格地址】
chart = book.add_chart({"type": "column"})
chart.set_title({"name": "Sales per Region"})
chart.add_series({"name": "=Sheet1!A11", 
                  "categories": "=Sheet1!B10:C10", 
                  "values": "=Sheet1!B11:C11"})
chart.add_series({"name": "=Sheet1!A12", 
                  "categories": "Sheet1!B10:C10", 
                  "values": "Sheet1!B12:C12"})
chart.set_x_axis({"name": "Regions"})
chart.set_y_axis({"name": "Sales"})
sheet.insert_chart("A15", chart)

# 关闭工作簿并在磁盘上创建文件
book.close()

In [16]:
book = xlsxwriter.Workbook("xl/macro_xlsxwriter.xlsm")
sheet = book.add_worksheet("Sheet1")
sheet.write("A1", "Click the button！")
book.add_vba_project("xl/vbaProject.bin")
sheet.insert_button("A3", {"macro": "Hello", 
                           "caption": "Button 1", 
                           "width":130, "height": 35})
book.close()

# pyxlsb
【单元格索引从1开始】

In [17]:
import pyxlsb
import excel

在pyxlsb中，工作簿和sheet对象都可以被用作上下文管理器。<br>
book.sheets返回的是工作表名称对象，而不是对象！<br>
要获取工作表对象，需要使用get_sheet()

In [18]:
# 遍历工作表
# sheet.dimension：获取工作表的维度——包括：dim.h: 工作表的总行数。dim.w: 工作表的总列数。
with pyxlsb.open_workbook("xl/stores.xlsb") as book:
    for sheet_name in book.sheets:
        with book.get_sheet(sheet_name) as sheet:
            dim = sheet.dimension
            print(f"Sheet '{sheet_name}' has " 
                  f"{dim.h} rows and {dim.w} cols")

Sheet '2019' has 7 rows and 5 cols
Sheet '2020' has 7 rows and 5 cols
Sheet '2019-2020' has 20 rows and 5 cols


In [19]:
# 利用excel模块读取一个区间中单元格的值，除了"2019"，也可以使用它的索引（从1开始）
with pyxlsb.open_workbook("xl/stores.xlsb") as book:
    with book.get_sheet("2019") as sheet:
        # 使用 excel 模块（自定义模块）从指定单元格（"B2"）开始读取数据
        data = excel.read(sheet, "B2")
# 打印前两行
data[:2]

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10.0, 'Sarah', 43301.0, False]]

pyxlsb目前无法识别包含日期的单元格，所以必须手动将以日期为格式的单元格中的值转换为datetime对象

In [20]:
from pyxlsb import convert_date
convert_date(data[1][3])

datetime.datetime(2018, 7, 20, 0, 0)

当使用版本低于1.3的pandas读取xlsb格式的文件时，需要显式地指定引擎

In [21]:
df = pd.read_excel("xl/stores.xlsb", engine="pyxlsb")

# xlrd、xlwt和xlutils 【用于xls文件】
【单元格索引从0开始】

1. 使用xlrd读取文件

In [22]:
import xlrd
import xlwt
from xlwt.Utils import cell_to_rowcol2
import xlutils
import excel

In [23]:
# 打开工作簿来读取单元格的值，在加载数据后文件会自动关闭
book = xlrd.open_workbook("xl/stores.xls")

In [24]:
# 获取所有工作表单的名称
book.sheet_names()

['2019', '2020', '2019-2020']

In [25]:
# 遍历所有工作表对象
for sheet in book.sheets():
    print(sheet.name)

2019
2020
2019-2020


In [26]:
# 通过名称或索引（从0开始）获取工作表对象
sheet = book.sheet_by_index(0)
sheet = book.sheet_by_name("2019")

In [27]:
# 维度（总行总列）
sheet.nrows, sheet.ncols

(8, 6)

*会解包cell_to_rowcol2返回的元组以生成各个参数<br>
cell_to_rowcol2("B3")：用于将 Excel 风格的单元格地址（如 "B3") 转换为行号和列号的元组 (row, col)；即"B3" 会被转换为 (2, 1)。<br>
*cell_to_rowcol2("B3"):用于将元组 (2, 1) 解包为两个独立的参数 2 和 1。

In [28]:
# 使用A1表示法或者单元格索引（从0开始）读取各个单元格的值
sheet.cell(*cell_to_rowcol2("B3")).value
sheet.cell(2, 1).value

'New York'

In [29]:
# 使用excel模块读取一个区间中单元格的值
data = excel.read(sheet, "B2")
# 打印前两行
data[:2]

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10.0, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

2. 使用xlwt写入文件【不能生成图表，只支持bmp格式的图片】

In [30]:
import xlwt
from xlwt.Utils import cell_to_rowcol2
import datetime as dt
import excel

在 xlwt 中，sheet.write()方法用于向工作表的特定单元格写入数据。这个方法通常需要三个参数：行号 (r)、列号 (c)、以及要写入的数据 (label)

In [31]:
# 实例化工作簿
book = xlwt.Workbook()

# 添加工作表并为其命名
sheet = book.add_sheet("Sheet1")

# 使用A1表示法和单元格索引（从0开始写入各个单元格）
sheet.write(*cell_to_rowcol2("A1"), "Hello 1")
sheet.write(r=1, c=0, label="Hello 2")

# 格式化：填充颜色、对齐、边框和字体
formatting =xlwt.easyxf("font: bold on, color red;"
                        "align: horiz center;"
                        "borders: top_color red, bottom_color red,"
                                "right_color red, left_color red,"
                                "left thin, right thin,"
                                "top thin, bottom thin;"
                        "pattern: pattern solid, fore_color yellow;")
sheet.write(r=2, c=0, label="Hello 3", style=formatting)

# 数字格式化（使用Excel的格式化字符串）
number_format = xlwt.easyxf(num_format_str="0.00")
sheet.write(3, 0, 3.3333, number_format)

# 日期格式化（使用Excel的格式化字符串）
date_format = xlwt.easyxf(num_format_str="mm/dd/yyyy")
sheet.write(4, 0, dt.datetime(2012, 2, 3), date_format)

# 公式：必须使用以逗号分隔的公式的英文名称
sheet.write(5, 0, xlwt.Formula("SUM(A4, 2)"))

# 二维列表（使用excel模块）
data = [[None, "North", "South"], 
        ["Last Year", 2, 5], 
        ["This Year", 3, 6]]
excel.write(sheet, data, "A10")

# 图片（只支持添加bmp格式的图片）
sheet.insert_bitmap("images/python.bmp", 0, 2)

# 将文件写入磁盘
book.save("xl/xlwt.xls")

3. 使用xlutils编辑文件

xlutils可以作为xlrd和xlwt之间的桥梁。<br>
这也表明了实际上这并不是真正的编辑操作：<br>
&nbsp;&nbsp;&nbsp;&nbsp;工作表通过xlrd读取包含格式在内的文件内容（将formatting_info的参数设置为True），然后再通过xlwt将期间作出的更改写入文件

In [32]:
import xlutils.copy

formatting_info参数是一个可选参数，它用于指示在打开Excel文件时是否加载格式信息。<br>
&nbsp;&nbsp;&nbsp;&nbsp;当formatting_info=True时，xlrd会尝试读取并加载Excel文件中的格式信息，如字体、颜色、边框、对齐方式等。这可能会增加内存消耗，因为需要存储更多的格式数据。<br>
&nbsp;&nbsp;&nbsp;&nbsp;当formatting_info=False时，xlrd不会加载格式信息，只读取数据内容。这通常可以节省内存，并且对于只需要数据而不需要格式的应用场景来说足够了。

In [33]:
book = xlrd.open_workbook("xl/stores.xls", formatting_info=True)
"""
使用xlutils.copy模块的copy 函数复制由 xlrd 打开的工作簿。
这一步是必要的，因为xlrd打开的工作簿是只读的，而 xlutils.copy 提供了一个可写的工作簿副本。
"""
book = xlutils.copy.copy(book)
book.get_sheet(0).write(0, 0, "changed!")
book.save("xl/stores_edited.xls")

# 8.2 读写包的高级主题

### 8.2.1. 处理大型Excel文件

两个问题：<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;读写的过程慢<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;计算机内存不足

##### 1. 使用OpenPyXL写入文件

write_only=True：可以让内存消耗保持在较低的水平<br>
这个参数会通过append()方法强制逐行写入，并且不再允许写入单个单元格

In [34]:
book = openpyxl.Workbook(write_only=True)
# 设置write_only=True参数后book.active就不可用了
sheet = book.create_sheet()
# 生成一张包含1000*200个单元格的工作表
for row in range(1000):
    sheet.append(list(range(200)))
book.save("xl/openpyxl_optimized.xlsx")

##### 2. 使用XlsxWriter写入文件

XlsxWriter有一个和OpenPyXL类似的选项：constant_memory<br>
constant_memory也会强制逐行写入<br>
需要以字典的形式来传递options参数

In [35]:
book = xlsxwriter.Workbook("xl/xlsxwriter_optimized.xlsx", 
                           options={"constant_memory": True})
sheet = book.add_worksheet()
# 生成一张包含1000*200个单元格的工作表
for row in range(1000):
    sheet.write_row(row, 0, list(range(200)))
book.close()

##### 3. 使用xlrd读取文件

on_demand=True：按需加载工作表

In [36]:
with xlrd.open_workbook("xl/stores.xls", on_demand=True) as book:
    # 只加载第一张工作表
    sheet = book.sheet_by_index(0)
sheet.name

'2019'

若不想将工作簿用作上下文管理器，则需手动调用book.release_resources()来正确关闭工作簿

In [37]:
# 搭配pandas在上下文管理器模式下使用xlrd
with xlrd.open_workbook("xl/stores.xls", on_demand=True) as book:
    with pd.ExcelFile(book, engine="xlrd") as f:
        df = pd.read_excel(f, sheet_name=0)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,Store,Employees,Manager,Since,Flagship
1,NaN,New York,10,Sarah,2018-07-20 00:00:00,False
2,NaN,San Francisco,12,Neriah,2019-11-02 00:00:00,MISSING
3,NaN,Chicago,4,Katelin,2020-01-31 00:00:00,NaN
4,NaN,Boston,5,Georgiana,2017-04-01 00:00:00,True
5,NaN,Washington DC,3,Evan,NaN,False
6,NaN,Las Vegas,11,Paul,2020-01-06 00:00:00,False


##### 4. 使用OpenPyXL读取文件

OpenPyXL读取大型Excel文件时需要控制内存，应该使用read_only=True参数加载工作簿<br>
由于OpenPyXL不支持with语句所以在工作完成时需要关闭文件<br>
若文件包含指向外部工作簿的链接，则需要使用keep_links=False参数加速读取过程<br>
keep_links：可以确保对外部工作簿的引用不会丢失；若只对读取某个工作簿的值感兴趣，那将这个参数设置为True可能会造成不必要的性能损失

In [38]:
book = openpyxl.load_workbook("xl/big.xlsx", 
                              data_only=True, 
                              read_only=True, 
                              keep_links=False)
# 在这里执行所需读取操作
book.worksheets[0].max_row, book.worksheets[0].max_column
book.close()  # 需设置参数read_only=True

##### 5. 并行读取工作表

pandas的read_excel()函数在读取大型工作簿的多张工作表时会花费很长时间【因为pandas会逐张读取工作表】<br>
若想过程更快，可使用<b>并行</b>读取这些工作表；可使用Python的多线程包实现<br>
多线程包：【系统有足够的内存处理工作负载的情况下】<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;多线程包会生成多个并行执行任务的Python解释器（通常一个CPU核心一个解释器）；即一个Python解释器处理一张工作表<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;但每个额外的Python解释器都需要一定时间启动并且需要占用额外的内存，所以文件小并行读取慢；而包含多个大型工作簿的大型文件，多线程则可以显著加快读取过程

In [39]:
import parallel_pandas
# sheet_name=None：读取Excel文件中的所有工作表。
# 如果你只想读取特定的工作表，可以提供一个工作表名称的列表作为sheet_name参数的值
parallel_pandas.read_excel("xl/stores.xlsx", sheet_name=["2019"])

{'2019':    Unnamed: 0     Unnamed: 1 Unnamed: 2 Unnamed: 3           Unnamed: 4  \
 0         NaN          Store  Employees    Manager                Since   
 1         NaN       New York         10      Sarah  2018-07-20 00:00:00   
 2         NaN  San Francisco         12     Neriah  2019-11-02 00:00:00   
 3         NaN        Chicago          4    Katelin  2020-01-31 00:00:00   
 4         NaN         Boston          5  Georgiana  2017-04-01 00:00:00   
 5         NaN  Washington DC          3       Evan                  NaN   
 6         NaN      Las Vegas         11       Paul  2020-01-06 00:00:00   
 
   Unnamed: 5  
 0   Flagship  
 1      False  
 2    MISSING  
 3        NaN  
 4       True  
 5      False  
 6      False  }

（单元格）魔法指令：%%time、%%timeit<br>
【必须出现在单元格的第一行，并且测量的是整个单元格的执行时间；若只想测量一行的执行时间，可以在行首使用】<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>%%time</b>：对比不同代码片段的执行时间<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>%%timeit（多核处理器上并行执行）</b>：运行单元格多次并取平均时间

In [40]:
%%time
data = pd.read_excel("xl/big.xlsx", 
                     sheet_name=None, 
                     engine="openpyxl")

CPU times: total: 55.1 s
Wall time: 55.1 s


In [41]:
%%time
import parallel_pandas
data = parallel_pandas.read_excel("xl/big.xlsx", 
                                  sheet_name=None)

CPU times: total: 109 ms
Wall time: 14.9 s


若只需读取单张大型工作表可使用Modin<br>
Modin在并行处理单张工作表的读取过程会实现显著的速度提升；但需要指定版本的pandas（可能会导致Anaconda附带的pandas被降级），可为此创建一个单独的Conda环境，以确保环境不会被弄乱

import modin.pandas
data = modin.pandas.read_excel("xl/big.xlsx", sheet_name=None, engine="openpyxl)

### 8.2.2 调整DataFrame在Excel中的格式

读取文件（pandas + OpenPyXL结合使用）

In [42]:
with pd.ExcelFile("xl/stores.xlsx", engine="openpyxl") as xlfile:
    # 读取DataFrame
    df = pd.read_excel(xlfile, sheet_name="2020")

    # 获取OpenPyXL工作簿对象
    book = xlfile.book
    
    # OpenPyXL代码从这里开始
    sheet = book["2019"]
    # 读取单个值
    value = sheet["B3"].value
value

'New York'

写入工作簿

In [43]:

with pd.ExcelWriter("xl/pandas_and_openpyxl.xlsx", engine="openpyxl") as writer:
    df = pd.DataFrame({"col1": [1, 2, 3, 4], 
                       "col2": [5, 6, 7, 8]})
    # 写入DataFrame
    df.to_excel(writer, "Sheet1", startrow=4, startcol=2)
    
    # 获取OpenPyXL工作簿和工作表对象
    book = writer.book
    sheet = writer.sheets["Sheet1"]

    # OpenPyXL的代码从这里开始
    # 写入单个单元格的值
    sheet["A1"].value = "This is a Title"

##### 1. 调整DataFrame索引和标题的格式

先创建DataFrame

In [44]:
df = pd.DataFrame({"col1": [1, -2], "col2": [-3, 4]}, 
                  index=["row1", "row2"])
df.index.name = "ix"
df

,col1,col2
ix,,
row1,1,-3
row2,-2,4


用OpenPyXL格式化索引和标题

In [45]:
from openpyxl.styles import PatternFill

In [46]:
with pd.ExcelWriter("xl/formatting_openpyxl.xlsx", engine="openpyxl") as writer:
    # 将整个df以默认格式从A1处写入
    df.to_excel(writer, startrow=0, startcol=0)
    
    # 将整个df以默认自定义索引/标题格式从A6处写入
    startrow, startcol = 0, 5
    # 1. 写入DataFrame的数据部分
    df.to_excel(writer, header=False, index=False, startrow=startrow+1, startcol=startcol+1)
    
    # 获取工作表对象并创建样式对象
    sheet = writer.sheets["Sheet1"]
    style = PatternFill(fgColor="D9D9D9", fill_type="solid")
    
    # 2. 写入带样式的列标题
    for i, col in enumerate(df.columns):
        sheet.cell(row=startrow+1, column=i+startcol+2, value=col).fill = style
    
    # 3.写入带样式的索引
    index = [df.index.name if df.index.name else None] + list(df.index)
    for i, row in enumerate(index):
        sheet.cell(row=i+startrow+1, column=startcol+1, value=row).fill = style

使用XlsxWriter对索引和标题进行格式化

In [47]:
# 使用XlsxWriter对索引和标题进行格式化
with pd.ExcelWriter("xl/formatting_xlsxwriter.xlsx", engine="xlsxwriter") as writer:
    # 将整个df以默认格式从A1处写入
    df.to_excel(writer, startrow=0, startcol=0)
    
    # 将整个df以默认自定义索引/标题格式从A6处写入
    startrow, startcol = 0, 5
    # 1. 写入DataFrame的数据部分
    df.to_excel(writer, header=False, index=False, startrow=startrow+1, startcol=startcol+1)
    # 获取工作簿对象和工作表对象并创建样式对象
    book = writer.book
    sheet = writer.sheets["Sheet1"]
    style = book.add_format({"bg_color": "#D9D9D9"})
    
    # 2. 写入带样式的标题
    for i, col in enumerate(df.columns):
        sheet.write(startrow, startcol+i+1, col, style)
    
    
    # 3. 写入带样式的索引
    index = [df.index.name if df.index.name else None] + list(df.index)
    for i, row in enumerate(index):
        sheet.write(startrow+i, startcol, row, style)

##### 2. 格式化DataFrame的数据部分

<i>OpenPyXL可以为每一个单元格应用一种格式</i><br>
<i>XlsxWriter只能对行或列应用格式</i>

使用OpenPyXL格式化DataFrame的数据部分

In [48]:
from openpyxl.styles import Alignment

In [49]:
with pd.ExcelWriter("xl/data_format_openpyxl.xlsx", engine="openpyxl") as writer:
    # 写入DataFrame
    df.to_excel(writer)
    
    # 获取工作簿对象和工作表对象
    book = writer.book
    sheet = writer.sheets["Sheet1"]
    
    # 格式化每一个单元格
    nrows, ncols = df.shape
    for row in range(nrows):
        for col in range(ncols):
            # 考虑到标题和索引，这里加1
            # 因为OpenPyXL的索引是从1开始的，所以还要加1
            cell = sheet.cell(row=row+2, 
                              column=col+2)
            cell.number_format = "0.000"
            cell.alignment = Alignment(horizontal="center")

使用XlsxWriter格式化DataFrame的数据部分

In [50]:
with pd.ExcelWriter("xl/data_format_xlsxwriter.xlsx", engine="xlsxwriter") as writer:
    # 写入DataFrame
    df.to_excel(writer)
    
    # 获取工作簿对象和工作表对象
    book = writer.book
    sheet = writer.sheets["Sheet1"]
    
    # 格式化各列
    number_format = book.add_format({"num_format": "0.000", 
                                     "align": "center"})
    sheet.set_column(first_col=1, last_col=2, cell_format=number_format)

pandas在DataFrame上提供了<b>实验性</b>的style属性作为一种替代品<br>
实验性：其语法随时可能发生改变<br>
由于样式属性是为了将DataFrame格式化为HTML而引入，因此他们使用的是CSS语法

In [51]:
# 需要通过applymap对Styler对象的每个元素应用一个函数。通过df.style属性可以获得Styler对象
df.style.applymap(lambda x: "number-format: 0.000;" "text-align: center").to_excel("xl/styled.xlsx")

在不依赖样式属性的情况下，pandas还提供了对日期和时间的格式化支持

In [52]:
df = pd.DataFrame({"Date": [dt.date(2020, 1, 1)], 
                "Datetime":[dt.datetime(2020, 1, 1, 10)]})
with pd.ExcelWriter("xl/date.xlsx", 
                    date_format="yyyy-mm-dd", 
                    datetime_format="yyyy-mm-dd hh:mm:ss") as writer:
    df.to_excel(writer)